In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
import os
from vocab import Vocabulary, deserialize_vocab  # NOQA
import evaluation
import data
import torch
from train import validate
from model import SCAN
from data import get_loaders
from tqdm import tqdm
import numpy 
import json 
from glob import glob
import numpy as np
from model import l2norm
from evaluation import encode_data_vector

/home/jonatas/.conda/envs/jonatas/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
model_path = '../temp/results_alpha/en/f30k/pc_gru_1/model_best.pth.tar'

In [4]:
# load model and options
checkpoint = torch.load(model_path)
opt = checkpoint['opt']
model = SCAN(opt)
model.load_state_dict(checkpoint['model'])

EncoderImagePrecomp(
  (fc): Linear(in_features=2048, out_features=1024, bias=True)
)
()
PartialGRU(
  (embed): PartialConcat(
    (embedding): EmbeddingLayer(
      (embedding): Embedding(128, 24)
    )
    (layers): Sequential(
      (0): Sequential(
        (0): Conv1d(624, 128, kernel_size=(1,), stride=(1,))
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
      )
      (1): Sequential(
        (0): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
      )
      (2): Sequential(
        (0): Conv1d(256, 300, kernel_size=(1,), stride=(1,))
        (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
      )
    )
  )
  (rnn): GRU(300, 1024, batch_first=True, bidirectional=True)
)


In [5]:
vocab = deserialize_vocab(os.path.join('../',opt.vocab_path))

In [6]:
with open('/opt/datasets/flickr/dataset_flickr30k.json', 'r') as fp:
    flickr = json.load(fp)
    f30k = {x['filename']: x for x in flickr['images']}

In [7]:
print('Loading dataset')
loader_en, loader_de = get_loaders(
    splits=['test', 'test'], 
    langs=['en', 'de'],
    data_name=opt.data_name, 
    vocab=vocab,
    batch_size=opt.batch_size, 
    workers=opt.workers, opt=opt,
)

Loading dataset


In [8]:
features = glob('/opt/datasets/flickr_features/pickles/*.pkl')
features = {fname.split('/')[-1][:-3]: fname for fname in features}

In [9]:
loader_en.dataset[idx]

NameError: name 'idx' is not defined

In [14]:
loader_en.dataset.captions[1]
print(len(loader_en.dataset.images))

5000


In [10]:
def get_instance(loader, idx):
    instance = loader.dataset[idx]
    instance = data.collate_fn_partial_inst(instance)
    img_id = loader.dataset.ids[idx]
    inst_dict = flickr['images'][img_id]
    doug_features = np.load(features[inst_dict['filename'][:-3]])
    return instance, inst_dict, doug_features

In [12]:
instance, f30k_info, doug_features = get_instance(loader_en, 1)
images, captions, lengths, ids = instance

In [16]:
f30k_info,doug_features

({u'filename': u'1007129816.jpg',
  u'imgid': 25,
  u'sentences': [{u'imgid': 25,
    u'raw': u'The man with pierced ears is wearing glasses and an orange hat.',
    u'sentid': 125,
    u'tokens': [u'the',
     u'man',
     u'with',
     u'pierced',
     u'ears',
     u'is',
     u'wearing',
     u'glasses',
     u'and',
     u'an',
     u'orange',
     u'hat']},
   {u'imgid': 25,
    u'raw': u'A man with glasses is wearing a beer can crocheted hat.',
    u'sentid': 126,
    u'tokens': [u'a',
     u'man',
     u'with',
     u'glasses',
     u'is',
     u'wearing',
     u'a',
     u'beer',
     u'can',
     u'crocheted',
     u'hat']},
   {u'imgid': 25,
    u'raw': u'A man with gauges and glasses is wearing a Blitz hat.',
    u'sentid': 127,
    u'tokens': [u'a',
     u'man',
     u'with',
     u'gauges',
     u'and',
     u'glasses',
     u'is',
     u'wearing',
     u'a',
     u'blitz',
     u'hat']},
   {u'imgid': 25,
    u'raw': u'A man in an orange hat starring at something.',
    

In [17]:
img_enc, text_enc, lens = model.forward_emb(images, captions, lengths=lengths)
img_enc = l2norm(img_enc, 2)
text_enc = l2norm(text_enc, 2)
img_enc.shape, text_enc.shape

(torch.Size([1, 36, 1024]), torch.Size([1, 14, 1024]))

In [18]:
attention = img_enc.bmm(text_enc.permute(0, 2, 1))

In [19]:
attention.shape

torch.Size([1, 36, 14])